# Load Dataset

In [1]:
import datasets 
from datasets import load_dataset

dataset = load_dataset("ag_news")

/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(dataset['train']), len(dataset['test'])

(120000, 7600)

In [4]:
import random
# randomly select 200 samples from the training set
random.seed(42)
sampled_dataset_idx = random.sample(range(len(dataset['train'])), 10000)
sampled_dataset = dataset['train'].select(sampled_dataset_idx)

In [5]:
# check the label distribution of sampled dataset
import pandas as pd
pd.Series(sampled_dataset['label']).value_counts()

1    2576
2    2483
0    2479
3    2462
Name: count, dtype: int64

In [6]:
# save the sampled dataset
sampled_dataset.save_to_disk('data/10000_sampled_agnews')

Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 194239.19 examples/s]


In [1]:
from utils import *
import llama
llama_config = ModelArguments()
llama_config.model = "meta-llama/Llama-2-7b-chat-hf"
llama_config.temperature = 0.3
llama_config.max_new_tokens = 100
llama_config.top_k = 5
llama_model = llama.LlamaModel(llama_config)

/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.59it/s]
/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Llama Model loaded


In [2]:
import datasets
dataset = datasets.load_from_disk("./data/10000_sampled_agnews")
from inference_agnews import *

In [7]:
from tqdm import tqdm
correct,total = 0,0
bar = tqdm(range(len(dataset['text'])))
with open('10000_sampled_agnews_llama.txt', 'w+') as f:
    for i in bar:
        pred = llama_model.predict(PROMPT.format(dataset[i]['text']))
        pred = postprocess(pred[0])
        f.write(str(pred) + '\n')
        if pred == dataset[i]['label']:
            correct += 1
        total += 1
        bar.set_description(f"Accuracy: {correct/total:.4f}")

Accuracy: 0.6866: 100%|██████████| 10000/10000 [2:22:19<00:00,  1.17it/s] 
